In [1]:
import requests 
import bs4 
import pandas as pd
import re
from requests.exceptions import MissingSchema

In [73]:
def get_info(url):
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, "html.parser").body
    
    try: accounts = soup.findAll('a', {'class':'twitter-handle'})[0].text
    except IndexError:  accounts = ''
    short_desc = soup.findAll('h5', {'class':None})[0].text
    scp = soup.findAll('p', {'class': None})
    des = '\n'.join([i.text for i in scp])
    tag = '\n'.join(find_tags(soup))
    name = soup.findAll('h1')[0].text.strip('\n').strip()
    
    return name, accounts, short_desc, des, tag

def find_tags(sp):
    collect = []
    for i in sp.findAll('a'):
        try: href = i['href']
        except KeyError: continue
        if 'tags_str' in href: collect.append(i.text)
    return collect

In [2]:
## get target urls
url = 'https://www.aei.org/our-scholars/'
r = requests.get(url)
soup = bs4.BeautifulSoup(r.text, "html.parser").body
asn = soup.findAll('a', {'class':'news-thumbnail'})
staff_urls = [i['href'] for i in asn]

In [74]:
names = []
twitter_accounts = []
short_descs = []
descs = []
tags = []

for i, link in enumerate(staff_urls):
    name, accounts, short_desc, des, tag = get_info(link)
    names.append(name)
    twitter_accounts.append(accounts)
    short_descs.append(short_desc)
    descs.append(des)
    tags.append(tag)
    print(i, end = '\r')

In [76]:
user_data = pd.DataFrame({'name': names,
                          'twitter':twitter_accounts,
                          'short_desc': short_descs,
                          'desc': descs,
                          'tag': tags})
user_data['twitter'] = user_data['twitter'].apply(lambda x: x.strip().strip('@'))
user_data = user_data.drop_duplicates()

In [14]:
user_data['twitter'] = user_data['twitter'].apply(lambda x: x.strip('@') if x != None else x)
user_data.to_csv('aei_twitter.csv', index=False)